In [ ]:
! pip install requests beautifulsoup4 certifi pyopenssl urllib3 html5lib pandas tqdm sqlalchemy psycopg2

In [ ]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from pandas import read_csv
from datetime import datetime
from tqdm import tqdm
from sqlalchemy import create_engine, URL
from urllib3.exceptions import InsecureRequestWarning
from pathlib import Path
import json
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

In [ ]:
article_list = [
    ("ดูแลลูกน้อย", 53),
    ("สุขภาพลูกรัก", 33),
    ("ประสบการณ์คุณแม่", 32),
    ("ดูแลครรภ์มารดา", 12),
    ("เตรียมตัวตั้งครรภ์", 10),
    ("lifestyle-คุณแม่", 3),
    ("คู่มือแม่ลูกอ่อน", 15),
    ("คู่มือแม่ลูกอ่อน/พัฒนาการทารกวัย-1-6-เดือน", 7),
    ("กินอิ่ม", 18),
    ("นอนหลับ", 10),
    ("รีวิวแม่และเด็ก", 7),
    ("ลูกดารา", 11),
    ("คำคมรักลูกนะ", 7),
    ("เตรียมตัวตั้งครรภ์/อาหารคุณแม่ตั้งครรภ์", 1),
    ("เตรียมตัวตั้งครรภ์/ตั้งครรภ์-40-สัปดาห์", 6),
    ("เตรียมตัวตั้งครรภ์/สุขภาพคุณแม่ตั้งครรภ์", 6),
    ("เตรียมตัวตั้งครรภ์/คุมกำเนิดหลังคลอด", 1),
    ("การคลอด", 4),
    ("คู่มือแม่ลูกอ่อน/พัฒนาการทารกวัย-6-12-เดือน", 6),
    ("eq-ลูกรัก", 2),
    ("คู่มือแม่ลูกอ่อน/พัฒนาการทารกวัย-1-6-เดือน", 7),
    ("คู่มือแม่ลูกอ่อน/พัฒนาการทารกวัย-12-18-เดือน", 3),
    ("คู่มือแม่ลูกอ่อน/พัฒนาการทารกวัย-2-3-ขวบ", 2),
    ("พัฒนาการเด็กโต", 1),
    ("สุขภาพลูกรัก/โรคในเด็ก", 11),
    ("สุขภาพลูกรัก/การป้องกันโรค", 24),
    ("เมนูลูกรัก/เมนูลูกรัก-วัย-6-9-เดือน", 23),
    ("เมนูลูกรัก/เมนูลูกรัก-วัย-9-12-เดือน", 8),
    ("เมนูลูกรัก/เมนูลูกรัก-วัย-12-18-เดือน", 1),
    ("เมนูลูกรัก/เมนูลูกรัก-วัย-2-ขวบ", 1),
    ("เมนูลูกรัก/เมนูเด็กโต", 2),
    ("โรงเรียนลูก", 1),
]


In [ ]:
article = []
for cat, max in article_list:
    for i in tqdm(range(1, max+1), desc=cat):
        data = requests.get(f"https://mumhelper.com/category/{cat}/page/{i}/", verify=False)
        soup = BeautifulSoup(data.content, "html5lib")
        for ar in soup("article"):
            article.append(ar.a["href"])


In [ ]:
with open("Mom_and_Baby.txt", "w") as f:
    for a in set(article):
        f.write(f"{a}\n")

---------

In [ ]:
with open("Mom_and_Baby.txt", "r") as f:
    article = list(map(str.strip, f.readlines()))

In [ ]:
db_data = {}
filename = "Mom_and_Baby_{i}.csv"

for i, url in tqdm(enumerate(article), total=len(article)):
    if i%100 == 0 and len(db_data) >= 100:
        df = DataFrame(db_data).transpose()
        df.to_csv(filename.format(i))
        db_data = {}

    try:
        data = requests.get(url, verify=False)
        soup = BeautifulSoup(data.content, "html5lib")

        paragraph = []
        for p in soup.find("div", {"class": "entry-content"})("p"):
            text = p.text.replace("\xa0", " ")
            if len(text) < 150:
                continue
            paragraph.append(text)
    
        db_data[url] = {
                "url": url,
                "category": "Mom & Baby",
                "title": soup.title.string,
                "content": None,
                "paragraphs": json.dumps(paragraph, ensure_ascii=False),
                "raw_html": soup,
                "raw_json": None,
                "remark": None,
                "created_at": datetime.now(),
            }
    except Exception as e:
        print(f"Exception on {url} by {e.args}")

df = DataFrame(db_data).transpose()
df.to_csv(filename.format(i+1))


In [ ]:
engine = create_engine()

In [ ]:
list_path = list(Path(".").glob("Mom_and_Baby_*.csv"))
for path in tqdm(list_path):
    df = read_csv(path)
    df.drop(columns=["Unnamed: 0"], inplace=True)
    df.to_sql("article_raw_data", engine, index=False, if_exists='append', method="multi")